In [109]:
import numpy as np
import nglview as nv
import MDAnalysis as mda

u = mda.Universe("../data/aldp/aldp.pdb")

date = "aldp_ours_3"
positions = np.load(f"../sample/{date}/samples.npy")[62].reshape(-1, 3)

new_u = mda.Merge(u.atoms)
new_u.atoms.positions = positions * 10

view = nv.show_mdanalysis(new_u)
view.add_representation("ball+stick")
view

NGLWidget()

In [111]:
view.download_image(filename=f'../figure/aldp_ours.png')

In [112]:
from PIL import Image

# Load the saved image
image_path = '../figure/aldp_ours.png'
image = Image.open(image_path)

# Crop the image to a square
width, height = image.size
min_dimension = min(width, height)
left = (width - min_dimension) // 2
top = (height - min_dimension) // 2
right = left + min_dimension
bottom = top + min_dimension

cropped_image = image.crop((left, top, right, bottom))

# Save the cropped image
cropped_image.save('../figure/aldp_ours_cropped.png')